In [2]:
import pandas as pd
import plotly.graph_objects as go
from signals import *

In [ ]:
# Can we reasample dataframes?
print(load_ticker("AUDCAD").head(2))
print(load_ticker("AUDCAD", 60).head(2))
# Yes!

In [ ]:
# Can we calculate the supertrend?
from supertrend.Indicators.Super_Trend import ST
print("Loading data")
df = load_ticker("AUDCAD", 60)

print("Calculating SuperTrend")
df_60_ST = ST(df, 3, 10)[["Open","High","Low","Close","ATR","SuperTrend","SuperTrendType"]]
display(df_60_ST)
# Yes!

In [6]:
# How does indexing work in a resampled df
# Signal comes in at 03:30, so the 03:00 candle has not finished yet, we should use the 
# indicator from the previous candle, 02:00
#display(pd.concat([df_60.loc[:"2019-10-29 03:30"][-2:],df_60.loc["2019-10-29 03:30":]])) # -> Starts at 04:00, is this what I want

def trailing_df(df, datetime_str):
    """
    Given a signal within a candle, this function returns a df starting with the most recent 
    complete candle.
    
    e.g.
    1 hr sampling. 03:30 signal time (3-4 candle not yet complete), returns starting with the 02:00 candle
    """  
    return pd.concat([df.loc[:datetime_str][-2:],df.loc[datetime_str:]])
    

In [7]:
def supertrend_trailing(df_full, signal):
    """
    Process signal using trailing supertrend stop loss.
    TODO: accound for spread and fees
    """
    #display(df_full)
    df = trailing_df(df_full, signal.datetime_str)
    #display(df)

    real_entry = signal.entry_price # Assume trade entered immediatley
    stop_loss = signal.stop_loss
    
    ohlc_values = df.iterrows()
    _, previous_ohlc = next(ohlc_values)
    
    previous_type = previous_ohlc.SuperTrendType
    previous_supertrend = previous_ohlc.SuperTrend
    

    for dt, ohlc in ohlc_values:
        if signal.type == "BUY":
            if previous_type == 1:
                if stop_loss < previous_supertrend:
                    stop_loss = previous_supertrend
            
            if ohlc.High >= signal.take_profit:
                return signal.take_profit - real_entry
            if ohlc.Low <= stop_loss:
                return stop_loss - real_entry
            
        if signal.type == "SELL":
            if previous_type == -1:
                if stop_loss > previous_supertrend:
                    stop_loss = previous_supertrend   
                    
            if ohlc.Low <= signal.take_profit:
                return real_entry - signal.take_profit
            if ohlc.High >= stop_loss:
                return real_entry - stop_loss
            
        previous_type = ohlc.SuperTrendType
        previous_supertrend = ohlc.SuperTrend
        
    return 0.0
        

In [130]:
beta_tp1 = Signal(ticker="NZDCAD", datetime_str="2019-12-04 12:44",
                  entry_price=0.8656, stop_loss=0.868, take_profit=0.8636)
df = get_supertrend(beta_tp1.ticker, 6, 40, 60)

print(beta_tp1, supertrend_trailing(df, beta_tp1) * pip_factor(beta_tp1.ticker))

NZDCAD: 2019-12-04 12:44: Entry: 0.8656, SL: 0.868, TP: 0.8636 20.000000000000018


In [27]:
signals = load_beta_trades("./beta/BETA.csv")
total_profit = 0.0
wins = 0
losses = 0
for signal in signals["TP1"]:
    signal.set_stop_loss_pips(42)
    signal.set_take_profit_pips(70)

    df = get_supertrend(signal.ticker, multiplier=6, length=40, sample_mins=60)
    profit = supertrend_trailing(df, signal) * pip_factor(signal.ticker)
    if profit > 0:
        wins += 1
    else:
        losses += 1
    total_profit += profit
    print(signal, profit)
print(total_profit, wins, losses)

AUDCAD: 2020-01-14 13:19: Entry: 0.902, SL: 0.9062, TP: 0.895 0.0
AUDNZD: 2020-01-14 13:16: Entry: 1.043, SL: 1.0472, TP: 1.036 0.0
CADJPY: 2020-01-14 13:13: Entry: 84.2, SL: 84.62, TP: 83.5 0.0
AUDUSD: 2020-01-14 13:08: Entry: 0.69, SL: 0.6941999999999999, TP: 0.6829999999999999 0.0
NZDJPY: 2020-01-14 13:06: Entry: 72.8, SL: 73.22, TP: 72.1 -42.00000000000017
XAUUSD: 2020-01-14 10:56: Entry: 1544.94, SL: 1540.74, TP: 1551.94 70.0
GBPUSD: 2020-01-13 14:06: Entry: 1.29843, SL: 1.29423, TP: 1.3054299999999999 69.99999999999895
XAUUSD: 2020-01-10 01:31: Entry: 1551.91, SL: 1556.1100000000001, TP: 1544.91 -42.000000000000455
GBPJPY: 2020-01-09 02:08: Entry: 143.174, SL: 143.594, TP: 142.47400000000002 69.99999999999886
USDCAD: 2020-01-06 13:03: Entry: 1.2969, SL: 1.2927, TP: 1.3038999999999998 69.99999999999895
NZDCAD: 2019-12-04 12:44: Entry: 0.8656, SL: 0.8698, TP: 0.8586 70.00000000000006
AUDJPY: 2019-12-04 12:43: Entry: 74.27, SL: 74.69, TP: 73.57 -26.74835961373816
CADJPY: 2019-12-04 

With SL, TP and Trailing loss
60mins
SL = 50 pips
TP = 70 pips
Length = 40
Multiplier = 6
profit = 1514 pips (30 wins, 22 losses)

30mins
SL = 50 pips
TP = 70 pips
Length = 40
Multiplier = 8
profit = 1430 pips (29 wins, 23 losses)

15mins
SL = 50 pips
TP = 70 pips
Length = 40
Multiplier = 13
profit = 1227 pips (27 wins, 25 losses)

In [ ]:
# Multiplier Analysis

for multiplier in [5, 8, 1]:
    signals = load_beta_trades("./beta/BETA.csv")
    total_profit = 0.0
    wins = 0
    losses = 0
    for signal in signals["TP1"]:
        signal.set_stop_loss_pips(50)
        signal.set_take_profit_pips(70)

        df = get_supertrend(signal.ticker, multiplier=multiplier, length=40, sample_mins=60)
        profit = supertrend_trailing(df, signal) * pip_factor(signal.ticker)
        if profit > 0:
            wins += 1
        else:
            losses += 1
        total_profit += profit
        # print(signal, profit)
    print(multiplier, total_profit, wins, losses)

In [23]:
# Length Analysis

for length in range(36, 49, 2):
    signals = load_beta_trades("./beta/BETA.csv")
    total_profit = 0.0
    wins = 0
    losses = 0
    for signal in signals["TP1"]:
        signal.set_stop_loss_pips(42)
        signal.set_take_profit_pips(70)
        df = get_supertrend(signal.ticker, multiplier=6, length=length, sample_mins=60)
        profit = supertrend_trailing(df, signal) * pip_factor(signal.ticker)
        if profit > 0:
            wins += 1
        else:
            losses += 1
        total_profit += profit
        #print(signal, profit)
    print(length, total_profit, wins, losses)

36 1448.8812779423856 29 23
38 1449.5623632740019 29 23
40 1450.3089597254834 29 23
42 1450.9782849930084 29 23
44 1451.7362583177596 29 23
46 1455.9794108877118 29 23
48 1365.9598131141654 28 24


In [20]:
# Stop Loss Analysis

multiplier = 6
length = 40
take_profit = 70

profits = []
stops = range(10, 100, 1)

for stop in stops:
    signals = load_beta_trades("./beta/BETA.csv")
    total_profit = 0.0
    wins = 0
    losses = 0
    for signal in signals["TP1"]:
        signal.set_stop_loss_pips(stop)
        signal.set_take_profit_pips(take_profit)

        df = get_supertrend(signal.ticker, multiplier=multiplier, length=length, sample_mins=60)
        profit = supertrend_trailing(df, signal) * pip_factor(signal.ticker)
        if profit > 0:
            wins += 1
        else:
            losses += 1
        total_profit += profit
        #print(signal, profit)
    profits.append(total_profit)
    print(stop, total_profit, wins, losses)
    
fig = go.Figure(data=go.Scatter(x=list(stops), y=profits))
fig.update_layout(
    title=f"BETA: Stop Loss Optimisation (TP = {take_profit}, Mul = {multiplier}, Range = {length})",
    title_x=0.5,
    xaxis_title="Stop Loss (pips)",
    yaxis_title="Profit (pips)"
)
fig.show()

10 528.4956041455492 13 39
11 590.3465577646738 15 37
12 719.34655776467 17 35
13 686.346557764684 17 35
14 821.3465577646798 19 33
15 975.3465577646757 21 31
16 947.3465577646796 21 31
17 920.088348351339 21 31
18 981.2752561368216 22 30
19 956.2752561368263 22 30
20 1111.275256136828 24 28
21 1088.275256136833 24 28
22 1157.4172769565573 25 27
23 1136.4172769565596 25 27
24 1210.1923425922605 26 26
25 1304.153243544527 27 25
26 1313.1532435445247 27 25
27 1297.9300232328856 27 25
28 1284.9300232328883 27 25
29 1271.9300232328842 27 25
30 1258.9300232328883 27 25
31 1245.9300232328824 27 25
32 1354.3089597254843 28 24
33 1377.3089597254834 28 24
34 1402.3089597254793 28 24
35 1394.3089597254848 28 24
36 1386.3089597254839 28 24
37 1378.3089597254798 28 24
38 1370.3089597254857 28 24
39 1362.3089597254816 28 24
40 1354.308959725482 28 24
41 1457.3089597254861 29 23
42 1450.3089597254834 29 23
43 1443.308959725483 29 23
44 1436.308959725483 29 23
45 1429.3089597254848 29 23
46 1422.3089

In [13]:
# Take Profit Analysis

multiplier = 6
length = 40
stop = 41

profits = []
take_profits = range(10, 200, 2)

for take_profit in take_profits:
    signals = load_beta_trades("./beta/BETA.csv")
    total_profit = 0.0
    wins = 0
    losses = 0
    for signal in signals["TP1"]:
        signal.set_stop_loss_pips(stop)
        signal.set_take_profit_pips(take_profit)

        df = get_supertrend(signal.ticker, multiplier=multiplier, length=length, sample_mins=60)
        profit = supertrend_trailing(df, signal) * pip_factor(signal.ticker)
        if profit > 0:
            wins += 1
        else:
            losses += 1
        total_profit += profit
        #print(signal, profit)
    profits.append(total_profit)
    print(take_profit, total_profit, wins, losses)
    
fig = go.Figure(data=go.Scatter(x=list(take_profits), y=profits))
fig.update_layout(
    title=f"BETA: Take Profit Optimisation (SL = {stop}, Mul = {multiplier}, Range = {length})",
    title_x=0.5,
    xaxis_title="Take Profit (pips)",
    yaxis_title="Profit (pips)"
)
fig.show()

6 307.7767796883377 47 5
6 367.91880050810914 46 6
6 404.9188005080949 45 7
6 494.91880050809493 45 7
6 584.9188005081076 45 7
6 674.9188005080932 45 7
6 764.9188005080842 45 7
6 820.8088531749643 44 8
6 882.8088531749781 43 9
6 968.8088531749704 43 9
6 922.306986536456 41 11
6 924.0487771231358 39 13
6 1002.0487771231457 39 13
6 1020.8238427588485 38 14
6 1096.8238427588356 38 14
6 1172.8238427588458 38 14
6 1248.8238427588337 38 14
6 1182.7046458899363 36 16
6 1254.704645889935 36 16
6 1134.6655468422066 33 19
6 1149.161150987732 32 20
6 1143.3480587732354 31 21
6 1205.348058773241 31 21
6 1267.3480587732383 31 21
6 1329.3480587732317 31 21
6 1391.3480587732238 31 21
6 1336.3089597254973 30 22
6 1394.3089597254889 30 22
6 1345.308959725487 29 23
6 1401.3089597254955 29 23
6 1457.3089597254861 29 23
6 1442.495867510988 29 23
6 1432.3656435551397 29 23
6 1340.4584806960356 29 23
6 1388.4584806960313 29 23
6 1276.1529108011805 28 24
6 1320.1529108011734 28 24
6 1239.1529108011782 27 25


In [ ]:
# Example plotly scatter plot
fig = go.Figure(data=go.Scatter(x=list(take_profits), y=profits))
fig.update_layout(
    title="BETA Take Profit Optimisation",
    title_x=0.5,
    xaxis_title="Take Profit Level (pips)",
    yaxis_title="Profit (pips)"
)
fig.show()

In [ ]:
# Example plotly candlestick plot
dfp = df.loc["2019-01-14 21:00":"2019-01-14 22:00"]
fig = go.Figure(data=[go.Candlestick(x=dfp.index,
                open=dfp['Open'],
                high=dfp['High'],
                low=dfp['Low'],
                close=dfp['Close'])])

fig.show()

In [ ]:
# How to pretty print a dataframe
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_60)